In [1]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1693735177826_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
Schema = StructType([StructField('year', IntegerType(), nullable = True),
                        StructField('month', StringType(), True),
                        StructField('day', IntegerType(), True),
                        StructField('weekday', StringType(), True),
                        StructField('hour', IntegerType(), True),
                        StructField('atm_status', StringType(), True),
                        StructField('atm_id', StringType(), True),
                        StructField('atm_manufacturer', StringType(), True),
                        StructField('atm_location', StringType(), True),
                        StructField('atm_streetname', StringType(), True),
                        StructField('atm_street_number', IntegerType(), True),
                        StructField('atm_zipcode', IntegerType(), True),
                        StructField('atm_lat', DoubleType(), True),
                        StructField('atm_lon', DoubleType(), True),
                        StructField('currency', StringType(), True),
                        StructField('card_type', StringType(), True),
                        StructField('transaction_amount', IntegerType(), True),
                        StructField('service', StringType(), True),
                        StructField('message_code', StringType(), True),
                        StructField('message_text', StringType(), True),
                        StructField('weather_lat', DoubleType(), True),
                        StructField('weather_lon', DoubleType(), True),
                        StructField('weather_city_id', IntegerType(), True),
                        StructField('weather_city_name', StringType(), True),
                        StructField('temp', DoubleType(), True),
                        StructField('pressure', IntegerType(), True),
                        StructField('humidity', IntegerType(), True),
                        StructField('wind_speed', IntegerType(), True),
                        StructField('wind_deg', IntegerType(), True),
                        StructField('rain_3h', DoubleType(), True),
                        StructField('clouds_all', IntegerType(), True),
                        StructField('weather_id', IntegerType(), True),
                        StructField('weather_main', StringType(), True),
                        StructField('weather_description', StringType(), True)])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df = spark.read.format("csv")\
                .option("inferSchema", "false")\
                .schema(Schema)\
                .option("header", "false")\
                .load("/user/hadoop/ETL/ATMDATA/part-m-00000")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_id: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: double (nullable = true)
 |-- atm_lon: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: double (nullable = true)
 |-- weather_lon: double (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather_city_name: st

In [5]:
df.select('*').count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [ ]:
 2468572

### Creating the Dimension and Fact tables
##### Creating a dataframe for Location Dimension according to Target Dimension Model

In [6]:
location = df.select("atm_location", "atm_streetname", "atm_street_number", "atm_zipcode", "atm_lat", "atm_lon").distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
location.select('*').count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

In [8]:
from pyspark.sql.functions import *

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# creating the primary key column
df_temp = location.rdd.zipWithIndex().toDF()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df_temp.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---+
|                  _1| _2|
+--------------------+---+
|[Kolding, Vejleve...|  0|
|[Skelagervej 15, ...|  1|
|[Intern HolbÃƒÂ¦k...|  2|
|[Odense, FÃƒÂ¦lle...|  3|
|[Ikast, RÃƒÂ¥dhus...|  4|
|[Randers, ÃƒËœste...|  5|
|[Esbjerg, Strandb...|  6|
|[Lyngby, Jernbane...|  7|
|[Skive, Adelgade,...|  8|
|[Durup, Torvet, 4...|  9|
|[Hadsund, Storega...| 10|
|[NÃƒÂ¦stved, Fari...| 11|
|[Hobro, Adelgade,...| 12|
|[Nordkraft, Kjell...| 13|
|[Slagelse, Marien...| 14|
|[Aars, Himmerland...| 15|
|[Fur, StenÃƒÂ¸re,...| 16|
|[Hasseris, Hasser...| 17|
|[SÃƒÂ¦by, Vesterg...| 18|
|[HelsingÃƒÂ¸r, Sc...| 19|
+--------------------+---+
only showing top 20 rows

In [11]:
dim_location = df_temp.select(col("_1.*"),col("_2").alias('location_id'))
dim_location.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-------------------+-----------------+-----------+-------+-------+-----------+
|    atm_location|     atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|location_id|
+----------------+-------------------+-----------------+-----------+-------+-------+-----------+
|         Kolding|           Vejlevej|              135|       6000| 55.505|  9.457|          0|
|  Skelagervej 15|        Skelagervej|               15|       9000| 57.023|  9.891|          1|
|Intern HolbÃƒÂ¦k|        Slotsvolden|                7|       4300| 55.718| 11.704|          2|
|          Odense|       FÃƒÂ¦lledvej|                3|       5000| 55.394|  10.37|          3|
|           Ikast|RÃƒÂ¥dhusstrÃƒÂ¦det|               12|       7430| 56.139|  9.154|          4|
+----------------+-------------------+-----------------+-----------+-------+-------+-----------+
only showing top 5 rows

In [12]:
DIM_LOCATION = dim_location.withColumnRenamed('atm_location','location')\
                            .withColumnRenamed('atm_streetname','streetname')\
                            .withColumnRenamed('atm_street_number','street_number')\
                            .withColumnRenamed('atm_zipcode','zipcode')\
                            .withColumnRenamed('atm_lat','lat')\
                            .withColumnRenamed('atm_lon','lon')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
DIM_LOCATION.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-------------------+-------------+-------+------+------+-----------+
|        location|         streetname|street_number|zipcode|   lat|   lon|location_id|
+----------------+-------------------+-------------+-------+------+------+-----------+
|         Kolding|           Vejlevej|          135|   6000|55.505| 9.457|          0|
|  Skelagervej 15|        Skelagervej|           15|   9000|57.023| 9.891|          1|
|Intern HolbÃƒÂ¦k|        Slotsvolden|            7|   4300|55.718|11.704|          2|
|          Odense|       FÃƒÂ¦lledvej|            3|   5000|55.394| 10.37|          3|
|           Ikast|RÃƒÂ¥dhusstrÃƒÂ¦det|           12|   7430|56.139| 9.154|          4|
+----------------+-------------------+-------------+-------+------+------+-----------+
only showing top 5 rows

In [14]:
atm = df.select('atm_id', 'atm_manufacturer', 'atm_lat', 'atm_lon')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
atm.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------------+-------+-------+
|atm_id|atm_manufacturer|atm_lat|atm_lon|
+------+----------------+-------+-------+
|     1|             NCR| 55.233| 11.763|
|     2|             NCR| 57.043|   9.95|
|     2|             NCR| 57.043|   9.95|
|     3|             NCR| 56.139|  9.154|
|     4|             NCR| 55.634| 12.018|
+------+----------------+-------+-------+
only showing top 5 rows

In [16]:
atm = atm.join(dim_location, on = ['atm_lat', 'atm_lon'], how = "left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
atm.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['atm_lat', 'atm_lon', 'atm_id', 'atm_manufacturer', 'atm_location', 'atm_streetname', 'atm_street_number', 'atm_zipcode', 'location_id']

In [19]:
atm = atm.select('atm_id', 'atm_manufacturer', 'location_id').distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
atm = atm.withColumnRenamed('location_id', 'atm_location_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
atm = atm.withColumnRenamed('atm_id', 'atm_number')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# creating the primary key column
df_temp = atm.rdd.zipWithIndex().toDF()
dim_atm = df_temp.select(col("_1.*"),col("_2").alias('atm_id'))
dim_atm.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------------+---------------+------+
|atm_number|atm_manufacturer|atm_location_id|atm_id|
+----------+----------------+---------------+------+
|        53|             NCR|             28|     0|
|        30|             NCR|             57|     1|
|        66|             NCR|             20|     2|
|        40| Diebold Nixdorf|             86|     3|
|        36|             NCR|             61|     4|
|        76|             NCR|             24|     5|
|        49|             NCR|            103|     6|
|        28|             NCR|             33|     7|
|        42|             NCR|             96|     8|
|         7| Diebold Nixdorf|             75|     9|
+----------+----------------+---------------+------+
only showing top 10 rows

In [23]:
dim_atm.show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------------+---------------+------+
|atm_number|atm_manufacturer|atm_location_id|atm_id|
+----------+----------------+---------------+------+
|        36|             NCR|             61|     0|
|        30|             NCR|             57|     1|
|        66|             NCR|             20|     2|
|        53|             NCR|             28|     3|
|        40| Diebold Nixdorf|             86|     4|
|        76|             NCR|             24|     5|
|        49|             NCR|            103|     6|
|        28|             NCR|             33|     7|
|        42|             NCR|             96|     8|
|         7| Diebold Nixdorf|             75|     9|
|        81|             NCR|             81|    10|
|       104|             NCR|             69|    11|
|        95|             NCR|             39|    12|
|        80|             NCR|             30|    13|
|        59| Diebold Nixdorf|             57|    14|
|        45|             NCR|             68| 

In [24]:
dim_atm.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

156

In [25]:
date = df.select('year', 'month', 'day', 'hour', 'weekday')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
date = date.withColumn('full_date', concat_ws('-', date.year, date.month, date.day))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
date = date.withColumn('full_time', concat_ws(':', date.hour, lit('00'), lit('00')))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
date = date.withColumn('full_date_time', concat_ws(' ', date.full_date, date.full_time))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
date = date.withColumn('full_date_time', unix_timestamp(date.full_date_time, 'yyyy-MMM-dd HH:mm:ss').cast('timestamp'))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
date.show(5, truncate = False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+----+-------+--------------+---------+-------------------+
|year|month  |day|hour|weekday|full_date     |full_time|full_date_time     |
+----+-------+---+----+-------+--------------+---------+-------------------+
|2017|January|1  |0   |Sunday |2017-January-1|0:00:00  |2017-01-01 00:00:00|
|2017|January|1  |0   |Sunday |2017-January-1|0:00:00  |2017-01-01 00:00:00|
|2017|January|1  |0   |Sunday |2017-January-1|0:00:00  |2017-01-01 00:00:00|
|2017|January|1  |0   |Sunday |2017-January-1|0:00:00  |2017-01-01 00:00:00|
|2017|January|1  |0   |Sunday |2017-January-1|0:00:00  |2017-01-01 00:00:00|
+----+-------+---+----+-------+--------------+---------+-------------------+
only showing top 5 rows

In [31]:
date = date.select('year', 'month', 'day', 'hour', 'weekday', 'full_date_time').distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
help(concat_ws)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Help on function concat_ws in module pyspark.sql.functions:

concat_ws(sep, *cols)
    Concatenates multiple input string columns together into a single string column,
    using the given separator.
    
    >>> df = spark.createDataFrame([('abcd','123')], ['s', 'd'])
    >>> df.select(concat_ws('-', df.s, df.d).alias('s')).collect()
    [Row(s='abcd-123')]
    
    .. versionadded:: 1.5

In [33]:
# creating the primary key column
df_temp = date.rdd.zipWithIndex().toDF()
DIM_DATE = df_temp.select(col("_1.*"),col("_2").alias('date_id'))
DIM_DATE.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+---+----+-------+-------------------+-------+
|year|   month|day|hour|weekday|     full_date_time|date_id|
+----+--------+---+----+-------+-------------------+-------+
|2017| January|  6|   8| Friday|2017-01-06 08:00:00|      0|
|2017| January| 16|  18| Monday|2017-01-16 18:00:00|      1|
|2017| January| 29|   8| Sunday|2017-01-29 08:00:00|      2|
|2017|February| 26|   8| Sunday|2017-02-26 08:00:00|      3|
|2017|   March| 14|   6|Tuesday|2017-03-14 06:00:00|      4|
+----+--------+---+----+-------+-------------------+-------+
only showing top 5 rows

In [34]:
DIM_DATE.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8685

Count matches with Validation document provided

Creating DIM_CARD_TYPE 

In [37]:
card_type=df.select('card_type').distinct()
card_type.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12

In [38]:
df_temp = card_type.rdd.zipWithIndex().toDF()
DIM_CARD_TYPE = df_temp.select(col("_1.*"),col("_2").alias('card_type_id'))
DIM_CARD_TYPE.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+
|           card_type|card_type_id|
+--------------------+------------+
|Visa Dankort - on-us|           0|
|  Mastercard - on-us|           1|
|         HÃƒÂ¦vekort|           2|
|            VisaPlus|           3|
|     Dankort - on-us|           4|
+--------------------+------------+
only showing top 5 rows

In [39]:
DIM_CARD_TYPE.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12

# Creating Fact table

In [53]:
df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+-------+----+----------+------+----------------+------------+-------------------+-----------------+-----------+-------+-------+--------+----------+------------------+----------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+--------------------+
|year|  month|day|weekday|hour|atm_status|atm_id|atm_manufacturer|atm_location|     atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|currency| card_type|transaction_amount|   service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|  temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main| weather_description|
+----+-------+---+-------+----+----------+------+----------------+------------+-------------------+-----------------+-----------+-------+-------+--------+----------+------------------+----------+------------+------------+-

In [54]:
fact_loc = df.withColumnRenamed('atm_location','location')\
    .withColumnRenamed('atm_streetname','streetname')\
    .withColumnRenamed('atm_street_number','street_number')\
    .withColumnRenamed('atm_zipcode','zipcode')\
    .withColumnRenamed('atm_lat','lat')\
    .withColumnRenamed('atm_lon','lon')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
fact_loc.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+-------+----+----------+------+----------------+----------+-------------------+-------------+-------+------+------+--------+----------+------------------+----------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+--------------------+
|year|  month|day|weekday|hour|atm_status|atm_id|atm_manufacturer|  location|         streetname|street_number|zipcode|   lat|   lon|currency| card_type|transaction_amount|   service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|  temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main| weather_description|
+----+-------+---+-------+----+----------+------+----------------+----------+-------------------+-------------+-------+------+------+--------+----------+------------------+----------+------------+------------+-----------+-----------+-------------

In [55]:
# joining the dfs
fact_loc = fact_loc.join(DIM_LOCATION, on = ['location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon'], how = "left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
fact_loc.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon', 'year', 'month', 'day', 'weekday', 'hour', 'atm_status', 'atm_id', 'atm_manufacturer', 'currency', 'card_type', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'location_id']

In [57]:
dim_atm.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['atm_number', 'atm_manufacturer', 'atm_location_id', 'atm_id']

In [58]:
fact_loc = fact_loc.withColumnRenamed('atm_id', 'atm_number').withColumnRenamed('location_id', 'atm_location_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
fact_loc = fact_loc.join(dim_atm, on = ['atm_number','atm_manufacturer','atm_location_id'], how = "left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
fact_loc.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['atm_number', 'atm_manufacturer', 'atm_location_id', 'location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon', 'year', 'month', 'day', 'weekday', 'hour', 'atm_status', 'currency', 'card_type', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'atm_id']

In [61]:
DIM_DATE.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['year', 'month', 'day', 'hour', 'weekday', 'full_date_time', 'date_id']

In [62]:
fact_date = fact_loc.join(DIM_DATE, on = ['year', 'month', 'day', 'hour', 'weekday'], how = "left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
fact_date.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['year', 'month', 'day', 'hour', 'weekday', 'atm_number', 'atm_manufacturer', 'atm_location_id', 'location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon', 'atm_status', 'currency', 'card_type', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'atm_id', 'full_date_time', 'date_id']

In [65]:
DIM_CARD_TYPE.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['card_type', 'card_type_id']

In [66]:
fact_CARD = fact_date.join(DIM_CARD_TYPE, on = 'card_type', how = "left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
fact_CARD.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['card_type', 'year', 'month', 'day', 'hour', 'weekday', 'atm_number', 'atm_manufacturer', 'atm_location_id', 'location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon', 'atm_status', 'currency', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'atm_id', 'full_date_time', 'date_id', 'card_type_id']

In [69]:
fact_CARD = fact_CARD.withColumnRenamed('atm_location_id', 'weather_loc_id')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
# selecting and arranging only the required columns according to the target model
FACT_ATM_TRANS = fact_CARD.select('atm_id', 'weather_loc_id', 'date_id', 'card_type_id', 
'atm_status', 'currency', 'service', 'transaction_amount', 'message_code', 'message_text', 'rain_3h', 
'clouds_all', 'weather_id', 'weather_main', 'weather_description')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
FACT_ATM_TRANS.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['atm_id', 'weather_loc_id', 'date_id', 'card_type_id', 'atm_status', 'currency', 'service', 'transaction_amount', 'message_code', 'message_text', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description']

In [72]:
FACT_ATM_TRANS.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

Count of Records in each stage - 2468572


# Pushing tables to S3 bucket

In [75]:
def pushtos3(table_name,file_name):
    table_name.coalesce(1).write.format('csv').option('header','false').save(f's3://awsbucketshoyeb/{file_name}', mode='overwrite')
    


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
pushtos3(DIM_LOCATION,'DIM_location')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
pushtos3(dim_atm,'dim_atm')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
pushtos3(DIM_DATE,'DIM_DATE')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
pushtos3(DIM_CARD_TYPE,'DIM_CARD_TYPE')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
pushtos3(FACT_ATM_TRANS,'FACT_ATM_TRANS')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…